In [1]:
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

In [2]:
DATA_PATH = "../data/WESAD/raw/S2/"

In [4]:
base_path = "../data/WESAD/raw/WESAD/S2/S2_E4_Data/"

eda = pd.read_csv(base_path + "EDA.csv", header=None).values.flatten()
bvp = pd.read_csv(base_path + "BVP.csv", header=None).values.flatten()
hr = pd.read_csv(base_path + "HR.csv", header=None).values.flatten()
temp = pd.read_csv(base_path + "TEMP.csv", header=None).values.flatten()
acc = pd.read_csv(base_path + "ACC.csv", header=None).values

print("EDA length:", len(eda))
print("ACC shape:", acc.shape)

EDA length: 31496
ACC shape: (251972, 3)


In [6]:
# 1) Read metadata rows
eda_meta = pd.read_csv(base_path + "EDA.csv", header=None, nrows=2)
eda_start_unix = float(eda_meta.iloc[0, 0])   # row 0
eda_fs = float(eda_meta.iloc[1, 0])           # row 1

In [7]:
# 2) Read signal from row 3 onwards
eda_data = pd.read_csv(base_path + "EDA.csv", header=None, skiprows=2)[0].values

In [8]:
# 3) Build time axis in seconds and datetime
n = len(eda_data)
t_rel = np.arange(n) / eda_fs              # seconds since start
t_abs = eda_start_unix + t_rel             # unix timestamps

eda_df = pd.DataFrame(
    {"eda": eda_data},
    index=pd.to_datetime(t_abs, unit="s", utc=True)
)
eda_df.index.name = "time"

eda_df.head()

,eda
time,
2017-05-22 07:15:25+00:00,0.000000
2017-05-22 07:15:25.250000+00:00,0.328021
2017-05-22 07:15:25.500000+00:00,0.410026
2017-05-22 07:15:25.750000+00:00,0.433090
2017-05-22 07:15:26+00:00,0.444622
